# String distances



In [ ]:
import nltk
import numpy as np
import pandas as pd
import editdistance

In [ ]:
words = nltk.corpus.words.words()

In [ ]:
pd.DataFrame(words).to_csv("words_nltk.csv")

## BK tree

### How to create a BK-tree

- take any word from your set and use it in as your root node. Add word to the tree based on their distance to the root.

- A subtree within the tree will be a node and all its downstream nodes and weights. We can represent this as a pair storing:
    - `(node: str, subtree_of_node: Dict[int, Tuple(str, ... )])`. 



### Example

- Let us consider we started a tree with the word set [**book**, **books**, **cake**] then my tree would look like this if I started by making the word **book**:

```
       1 
book ----- books
    |
    | 4
    ----cake
```

We can represent this tree in memory as a pair 
`(Node, Subtree below Node)`. That is:

```
('book', {1: ('books', {}), 4: ('cake', {})})
```


- Add **boo**. Now we want to add the word **boo** which is at distace 1 from book. Notice though that there is already **book** at distance 1. The BK tree has to respect that every node have all children with different distances. If there is a colission (like we have now) then the new word must become a children of the node with which there was the colition. In this case, a children of **book**. The new weight from **books** to **boo** has to be `edit_dis(books,boo)=2`.

```
       1           2
book ----- books ----- boo
    |
    | 4
    ----cake
```

#### Adding [**cape**,**cart**,**boon**,**cook**].

- Add **cape**. Notice `edit_dis(books,cape)=4` but **cake** is altready at distance 4. Therefore **cape** needs to be a child of **cake** with a weighted edge of weight `edit_dis(cake,cape)=1`.
```
       1           2
book ----- books ----- boo
    |
    | 4        1  
    ----cake ----- cape
```

- Add **cart**. Notice `edit_dis(books,cart)=4`. Therefore **cart** needs to be a children of **cake** (or any subchildren of cake). Since `edit_dis(cake,cart)=2` then :
```
       1           2
book ----- books ----- boo
    |
    | 4        1  
    ----cake ----- cape
           |
           | 2        
           ----- cart
```

- Add **boon**. Notice  `edit_dis(book,boon)=1`. Therefore **boon** has to be a descendant of **books** (because books is a son of book with weight 1). Notice `edit_dis(books,boon)=2` but there is already node **boo** at distance 2. Therefore **boon** needs to be a descendant of **boo**. Notice `edit_dis(boo,boon)=1`.

```
       1           2         1
book ----- books ----- boo ----- boon
    |                   
    | 4        1  
    ----cake ----- cape
           |
           | 2        
           ----- cart
```

- Add **cook**.  Notice  `edit_dis(book,cook)=1` but book contains a descendant at distance 1, books, so cook has to be a dascendant of books.  Notice  `edit_dis(books,cook)=2` but books has a descendant at distance 2, **boo**, so cook has to be a descendant of boo.   Notice  `edit_dis(boo,cook)=2` and boo has no other descendant at distance 2. Therefore we can add cook as descendant of boo with weight 2.

```
       1           2         1
book ----- books ----- boo ----- boon
    |                    |
    |                    |  2
    |                    ------ cook
    | 4        1  
    ----cake ----- cape
           |
           | 2        
           ----- cart
```


## How to code a BK tree

Now we would like to know how do we construct a bk-tree in python and store it. A natural way to store a tree is using many tuples concatenated. Since a tree 

##### Exercise: fill in the BKTree class to create a tree

In [ ]:
words[0:10]

In [ ]:
class BKTree:
    def __init__(self, distfn, words):
        self.distfn = distfn

        it = iter(words)
        root = next(it)
        self.tree = (root, {})

        for w in it:
            self._add_word(self.tree, w)

    def _add_word(self, parent, word):
        pword, children = parent
        d = self.distfn(word, pword)
        
        # TODO BEGIN ------------------------------------------
        # call _add_word recursively to keep adding words

        # TODO END --------------------------------------------
        

In [ ]:
t = BKTree(editdistance.eval, ['book'])
t.tree

In [ ]:
t = BKTree(editdistance.eval, ['book', 'books'])
t.tree

In [ ]:
t = BKTree(editdistance.eval, ['book', 'books','cake'])
t.tree

In [ ]:
words_simple = ['book', 'books', 'cake', 'boo'] 
t = BKTree(editdistance.eval, words_simple)
t.tree

In [ ]:
words_simple = ['book', 'books', 'cake', 'boo', 'cape', 'cart', 'boon', 'cook'] 
t = BKTree(editdistance.eval, words_simple)
t.tree

### Building a BK-Tree with many words

Let us see the cost of constructing the tree for `words` which has 23k strings

In [ ]:
%%time
t = BKTree(editdistance.eval, [w.lower() for w in words])

In [ ]:
len(t.tree), len(t.tree[1]), type(t.tree[1])

In [ ]:
t.tree[1].keys()

In [ ]:
t.tree[1][12]

In [ ]:
t.tree[1][20]

In [ ]:
branches_from_root = t.tree[1]
branch_20          = branches_from_root[20]
branch_21          = branches_from_root[21]

In [ ]:
branch_20

In [ ]:
branch_21

## Searching in a BK tree

Now in order to search all words that appear at distance less or equal than a tolerance T form a query word `q` we need to visit all nodes n that are at distance D(n,q)+-N.

Let us consider word q=caqe, T=1, candidates = [], search=[book]

Select candidate **book** from search=[book]
- edit_dist(book,caqe) = 4 => candidates is not updated
    - Crawl all children of book at distance I=[4-1,4+1]=[3,5]
    - There is a single node, search=[book,cake], inside I.
    - search = [book,cake]\book = [cake]
    
Select candidate **cake** from search=[cake]
- edit_dist(cake,caqe) = 1 => candidates =[cake]
    - Crawl all children of book at distance I=[1-1,1+1]=[0,2]
    - There are 2 possible nodes, search=[cape, cart]


Select candidate **cape** from search=[cape,cart]
- edit_dist(cape,caqe) = 1 => candidates =[cake, cape]
    - Crawl all children of cape at distance I=[1-1,1+1]=[0,2]
    - Cape has no children
    - search = [cape, cart]\cape = [cart]
    
    
Select candidate **cart** from search=[cart]
- edit_dist(cart,caqe) = 2 => candidates is not updated
    - Crawl all children of cape at distance I=[1-1,1+1]=[0,2]
    - Caqe has no children
    - search = [cart]\cart = []


Select candidate... There is no candidate! stop process. 

The resulting set of possible candidates at distance 1 are: [cape,cake].

Notice that we ended up computing 4 edit distances yet we have 8 nodes.



### Making queries 

In [ ]:
visited_nodes = []
class BKTree:
    def __init__(self, distfn, words):
        self.distfn = distfn

        it = iter(words)
        root = next(it)
        self.tree = (root, {})

        for i in it:
            self._add_word(self.tree, i)

    def _add_word(self, parent, word):
        pword, children = parent
        d = self.distfn(word, pword)
        # TODO BEGIN ------------------------------------------

        # TODO END --------------------------------------------
        
    def _search_descendants(self, parent, n, query_word):   
        node_word, children_dict = parent
        dist_to_node = self.distfn(query_word, node_word)
        visited_nodes.append(node_word)
        results = []
        if dist_to_node <= n:
            results.append((dist_to_node, node_word))
        
        # TODO BEGIN ------------------------------------------
        # (dist - T, dist+T+1)

        # TODO END --------------------------------------------
          
        return results
            
    def query(self, query_word, n):
        # sort by distance
        return sorted(self._search_descendants(self.tree, n, query_word))

In [ ]:
%%time
t = BKTree(editdistance.eval, words)

In [ ]:
%%time
visited_nodes = []
print(t.query("senzorial", 2))

In [ ]:
len(visited_nodes), len(words)

If the edit distance is slow, the construction and search in the bktree will be slow.

Compare it with the nltk.edit_distance

In [ ]:
%%time
t_nltk = BKTree(nltk.distance.edit_distance, words)

In [ ]:
%%time
t_nltk.query("senzorial", 2)

In [ ]:
%%time
t.query("senzorial", 2)

### Comparing exhaustive computation vs bktree with the same edit distance function

In [ ]:
from typing import List

def get_candidates_exhaustive(distfn, query, max_dist, vocabulary, sort_candidates=True):
    """
    Compute the candidate words from the vocabulary at most `max_dist` away from the input token.
    Then it filters candidates by the computed values in the distance function.
    This function is mainly used for benchmarking purposes.
    :return: list of candidate words.
    """
    query = query.lower()
    distance_token_to_words = {
        word: distfn(query, word) for word in vocabulary
    }
    min_dist = min(distance_token_to_words.values())
    if min_dist <= max_dist:
        if sort_candidates:
            result = sorted(
                [
                    (distance, word)
                    for word, distance in distance_token_to_words.items()
                    if distance <= max_dist
                ]
            )
        else:
            result = [
                word
                for word, distance in distance_token_to_words.items()
                if distance <= max_dist
            ]
        return result

In [ ]:
word = "anthropomorphologicaly"
max_dist = 2
sort_candidates=False
        
%timeit candidates_ext = get_candidates_exhaustive(editdistance.eval, word, max_dist, words)

In [ ]:
candidates_ext = get_candidates_exhaustive(editdistance.eval, word, max_dist, words)
candidates_ext

In [ ]:
word = "anthropomorphologicaly"

%timeit candidates = t.query(word, 2)

In [ ]:
candidates = t.query(word, 2)
candidates

In [ ]:
word = "astrologi"
max_dist = 2
sort_candidates=False
        
%timeit candidates_ext = get_candidates_exhaustive(editdistance.eval, word, max_dist, words)

In [ ]:
word = "astrologi"

%timeit t.query(word, 2)

Notice that we have an issue with "hamer" since it is (in edit distance) as close as "hammer"  than "haler".

We would like to, somehow, choose wisely among a set of candidates.

In [ ]:
candidates_bktree = t.query("hamer", 1)
candidates_bktree

In [ ]:
%timeit  get_candidates_exhaustive(editdistance.eval, word, max_dist, words, sort_candidates)
%timeit  t.query(word, 1)

### Correcting a phrase

Notice that there are several words at the same distance. Picking the correct word is not the aim for today's class. The aim is to find a small list that contains the correct word.

In [ ]:
phrase = "the man wentt to the antimonarchik protest because he did not like the king"

In [ ]:
def correct_exhaustive(phrase, words):
    correction = []
    for w in phrase.split(" "):
        if w in words:
            correction.append(w)
        else:
            w_similar = get_candidates_exhaustive(editdistance.eval, w, 2, words, True)
            if len(w_similar)>0:
                w_corrected = w_similar[0][1]
                correction.append(w_corrected)
            else:
                # no word found, simply append the unedited word
                correction.append(w)
    return correction

In [ ]:
print("Original:\n", phrase)
print("Corrected:\n", " ".join(correct_exhaustive(phrase, words)))

In [ ]:
def correct_bktree(bktree, phrase, words):
    correction = []
    for w in phrase.split(" "):
        if w in words:
            correction.append(w)
        else:
            w_similar = bktree.query(w,2)
            if len(w_similar)>0:
                w_corrected = w_similar[0][1]
                correction.append(w_corrected)
            else:
                # no word found, simply append the unedited word
                correction.append(w)
    return correction

In [ ]:
print("Original:\n",phrase)
print("Corrected:\n"," ".join(correct_bktree(t, phrase, words)))

In [ ]:
%timeit correct_exhaustive(phrase, words)
%timeit correct_bktree(t, phrase, words)